In [24]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_images(images, titles):
    n = len(images)
    fig, axs = plt.subplots(1, n, figsize=(15, 5))
    if n == 1:
        axs = [axs]
    for i in range(n):
        axs[i].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB) if images[i].ndim == 3 and images[i].shape[2] == 3 else images[i], cmap='gray')
        axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.show()


def fundus_roi(image, mask=None):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    threshold = 15
    _, roi = cv2.threshold(v, threshold, 1, cv2.THRESH_BINARY)
    roi = roi.astype(np.uint8)
    white_pixels = np.argwhere(roi == 1)
    if white_pixels.size == 0:
        print("Aucun pixel blanc trouvé dans le masque.")
        return {"roi": roi, "diameter": 0, "image": image}
    x_min, y_min = np.min(white_pixels, axis=0)
    x_max, y_max = np.max(white_pixels, axis=0)
    diameter_x = x_max - x_min
    diameter_y = y_max - y_min
    diameter = int(np.maximum(diameter_x, diameter_y))
    return {"roi": roi, "diameter": diameter, "image": image}

def brightness_control(image, g=None, roi=None, diameter=None):
    adjusted_image = np.where(image + g <= 255, image + g, image)
    adjusted_image = np.where(image - g >= 0, image - g, image)
    return {"image": adjusted_image}

def negative_transformation(image):
    L = np.max(image)  
    negative_image = L - 1 - image
    return {"image":negative_image} 

def thresholding(image, threshold_value):
    _, thresholded_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return {"image":thresholded_image}

def apply_orthy(image):
    images = []
    titles = []

    data = fundus_roi(image)
    images.append(data["image"])
    titles.append('Original Image')

    #Brightness control
    g = 150
    brightness_control_result = brightness_control(**data, g)
    images.append(brightness_control_result["image"])
    titles.append('Brightness Control')
    #Negative Transformation
    negative_transformation_result = negative_transformation(**brightness_control_result)
    images.append(negative_transformation_result["image"])
    titles.append('Negative Transformation')
    #Thresholding
    threshold_value = 110
    thresholding_result = thresholding(**negative_transformation_result, threshold_value)
    images.append(thresholding_result["image"])
    titles.append('Thresholding')
    return thresholding_result



# Chargement de l'image
image = cv2.imread('../data/image_1.png')
if image is None:
    print("Erreur: Impossible de charger l'image.")
else:
    image = image.astype(np.uint8)
    orthy_image = apply_orthy(image)
    if orthy_image is not None:
        plot_images([orthy_image], ['Final Preprocessed Image'])



SyntaxError: positional argument follows keyword argument unpacking (3948620478.py, line 58)